In [7]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import requests
from datetime import datetime as dt
import warnings
from datetime import date
warnings.filterwarnings('ignore')

In [8]:
# loading Crypto price from api 
prices = pd.DataFrame()
prices['index'] = 1
asset = ['BTC','ETH','LTC','XRP']
prices_df2 = pd.DataFrame()

for i in range (0,4):
    
    url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={asset[i]}&market=USD&apikey=G0XWLJPTUJV8PAE5=json&outputsize=full"
    r = requests.get(url)
    r = r.json()
    prices_df = pd.DataFrame(r['Time Series (Digital Currency Daily)'])#getting daily btc price 
    prices_df = pd.DataFrame(prices_df.T[['4b. close (USD)']])# close price for btc
    #name = 'BTC'
    #prices_df.rename(columns={'4b. close (USD)': f"Close Price {asset[i]} "}, inplace = True)
    prices_df.reset_index(inplace = True)
    prices_df2[asset[i]] = list(prices_df['4b. close (USD)'])
    

In [9]:
prices_df2['Date']=prices_df['index']
crypto_df = prices_df2.set_index(['Date','BTC','ETH','LTC','XRP'])
crypto_df.reset_index(inplace = True)

In [10]:
crypto_df.to_csv('crypto.csv')

In [5]:
load_dotenv()

True

In [11]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [12]:
timeframe = "1D"
today = date.today()

In [13]:
start_date = pd.Timestamp("2018-09-18",tz="America/New_york").isoformat()
end_date = pd.Timestamp(today,tz="America/New_york").isoformat()
tickers = ["SPY"]
timeframe = "1D"
df_ticker = api.get_barset(
            tickers,
            timeframe,
            start = start_date,
            end = end_date,
            limit = 1000,
).df
df_ticker.head()

SPY                                   
                             open    high      low   close    volume
2018-09-18 00:00:00-04:00  289.57  291.58  289.550  290.90  38573521
2018-09-19 00:00:00-04:00  290.97  291.69  290.825  291.25  34596543
2018-09-20 00:00:00-04:00  292.64  293.94  292.510  293.55  54097569
2018-09-21 00:00:00-04:00  293.09  293.22  291.810  292.01  45805544
2018-09-24 00:00:00-04:00  291.34  291.50  290.370  291.07  37585070

In [14]:
# get level way
new_df = df_ticker[("SPY","close")].to_frame()
new_df.index = new_df.index.strftime("%Y-%m-%d")
new_df.columns = new_df.columns.get_level_values(0)
new_df

,SPY
2018-09-18,290.90
2018-09-19,291.25
2018-09-20,293.55
2018-09-21,292.01
2018-09-24,291.07
...,...
2021-06-09,421.62
2021-06-10,423.58
2021-06-11,424.31
2021-06-14,425.29


In [15]:
#drop index
new_df = df_ticker[("SPY","close")].to_frame()# if you grab one column it count as series but change to frame
new_df.index = new_df.index.strftime("%Y-%m-%d")# change to Date
new_df.columns = new_df.columns.droplevel()# with this column drop one level 
new_df.reset_index(inplace=True)#reset the index
new_df.columns = ['Date', 'SPY']# change the colume name
new_df.to_csv('spy.csv')
#df = new_df['close']
#df.head()  = 

In [17]:
Final_df = pd.merge(crypto_df, new_df, how='inner', on = 'Date')
Final_df.to_csv('Data.csv')